In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_dirty=pd.read_excel('/content/single sy.xlsx')
df_clean=pd.read_excel('/content/single sy.xlsx')

In [ ]:
df_dirty.head()

In [ ]:
df_dirty['Name']=df_dirty['Name'].str.lower()
df_clean['Name']=df_clean['Name'].str.lower()

In [ ]:
def get_matches_df(sparse_matrix, A, B, top=100):
    non_zeros = sparse_matrix.nonzero()

    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]

    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size

    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)

    for index in range(0, nr_matches):
        left_side[index] = A[sparserows[index]]
        right_side[index] = B[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]

    return pd.DataFrame({'left_side': left_side,
                         'right_side': right_side,
                         'similairity': similairity})

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def ngrams(string, n=3):
#     string = fix_text(string) # fix text encoding issues
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower() #make lower case
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string) #remove the list of chars defined above
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
#     string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single space
    string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [ ]:
import re as re
import time

In [ ]:
!pip3 install sparse_dot_topn

In [ ]:
from sparse_dot_topn import awesome_cossim_topn

Either select option one or two. Option one has numbers in the code ie; str[:-4], this is a command I added because i attached fixed random numbers to a to names when doing reconciliation to enable me identify the actual person bacause there are multiple people with the same names in the database.


Option 1

In [ ]:
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix_clean = vectorizer.fit_transform(df_clean['Name'].str[:-4].values.astype('U'))
tf_idf_matrix_dirty = vectorizer.transform(df_dirty['Name'].str[:-4].values.astype('U'))

t1 = time.time()
matches = awesome_cossim_topn(tf_idf_matrix_dirty, tf_idf_matrix_clean.transpose(), 3, 0)
t = time.time()-t1
print("SELFTIMED:", t)

matches_df = get_matches_df(matches, df_dirty['Name'], df_clean['Name'], top=0)

Option 2

In [ ]:
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix_clean = vectorizer.fit_transform(df_clean['Name'].values.astype('U'))
tf_idf_matrix_dirty = vectorizer.transform(df_dirty['Name'].values.astype('U'))

t1 = time.time()
matches = awesome_cossim_topn(tf_idf_matrix_dirty, tf_idf_matrix_clean.transpose(), 3, 0)
t = time.time()-t1
print("SELFTIMED:", t)

matches_df = get_matches_df(matches, df_dirty['Name'], df_clean['Name'], top=0)